In [10]:
import tensorflow as tf
import numpy as np

In [2]:
y_true = tf.constant([[0., 1.], [0., 0.]])
y_pred = tf.constant([[1., 1.], [1., 0.]])

2022-09-09 20:38:32.737227: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
loss_val = (tf.math.log(y_true + 1) - tf.math.log(y_pred + 1)) ** 2
print(tf.reduce_sum(loss_val) / 4)

tf.Tensor(0.2402265, shape=(), dtype=float32)


In [20]:
msle = tf.keras.losses.MeanSquaredLogarithmicError()
msle(y_true, y_pred).numpy()

0.24022643

In [41]:
a_loss = loss_val.numpy()[0,0]
a_loss = 0.13

diff = np.e ** np.sqrt(a_loss)
print(diff)

1.4341253160944203


In [39]:
msle([10000.], [11000.]).numpy()

0.009082219

## Keras

In [1]:
import tensorflow as tf
import numpy as np

2022-12-12 22:20:14.562886: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
x = tf.keras.layers.Input(shape=(1,), dtype=tf.float32)
y1 = tf.keras.layers.Dense(units=1, name='y1')(x)
y2 = tf.keras.layers.Dense(units=1, name='y2')(y1)

inner_model = tf.keras.models.Model(
    inputs={'x': x}, 
    outputs={'y1': y1, 'y2': y2},
    name="inner_model"
)

In [13]:
yy = inner_model(inner_model.input)

outputs = {name: tf.keras.layers.Lambda(lambda x: x, name=name)(value) for name, value in yy.items()}
outer_model = tf.keras.models.Model(
    inputs=inner_model.input, 
    outputs=outputs,
    name="outer_model"
)

In [14]:
train_ds = tf.data.Dataset.range(10).map(lambda x: ({'x': x}, {'y1': x+1, 'y2': x+2}))
batched = train_ds.batch(batch_size=2)
under_test = outer_model
under_test.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss={'y1': 'mse', 'y2': 'mse'}
)
under_test.fit(batched)

5/5 [==============================] - 1s 3ms/step - loss: 12.4593 - y1_loss: 0.2952 - y2_loss: 12.1641
